In [1]:
import psycopg2
import pandas as pd

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "***REMOVED***",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

In [5]:
create_table_query = '''CREATE TABLE EXPERIMENTS
    (EXPERIMENT_ID SERIAL PRIMARY KEY,
    DATASET TEXT NOT NULL,
    X REAL ARRAY NOT NULL,
    Y REAL ARRAY NOT NULL,
    VECTORIZER TEXT NOT NULL,
    IS_BINARY BOOLEAN NOT NULL,
    MIN_DF REAL NOT NULL,
    DISTANCE TEXT NOT NULL,
    K INTEGER NOT NULL,
    METHOD TEXT NOT NULL,
    MODEL  bytea NOT NULL,
    CLUSTERING_TIME INTEGER NOT NULL,
    GAP REAL NOT NULL,
    GAP_STD REAL NOT NULL,
    GAP_TIME REAL NOT NULL,
    SILHOUETTE REAL NOT NULL,
    SILHOUETTE_SAMPLES REAL ARRAY NOT NULL,
    SILHOUETTE_TIME REAL NOT NULL,
    COHERENCE_SAMPLES REAL ARRAY NOT NULL,
    COHERENCE_MED REAL NOT NULL,
    COHERENCE_STD REAL NOT NULL,
    COHERENCE_TIME REAL NOT NULL,
    COHERENCE_K INTEGER NOT NULL)
    '''

cursor.execute(create_table_query)
connection.commit()

In [31]:
cols = ['dataset', 'k', 'method', 'distance', 'clustering_time', 'gap_time', 'silhouette_time', 'coherence_time']
select_query = '''SELECT %s FROM EXPERIMENTS''' % ", ".join(cols)
cursor.execute(select_query)
r = cursor.fetchall()
# print(r)

In [32]:
df = pd.DataFrame(r, columns=cols)

In [33]:
df['total_time'] = df.sum(axis=1)

In [34]:
df

,dataset,k,method,distance,clustering_time,gap_time,silhouette_time,coherence_time,total_time
0,bbc5,2,nmf,euclidean,1,172.734,0.224299,8.53991,184.498209
1,bbc5,2,nmf,cosine,1,174.441,0.230242,8.89227,186.563512
2,bbc5,2,nmf,correlation,1,175.350,1.610340,8.53062,188.490960
3,bbc5,3,nmf,euclidean,1,176.075,0.222425,5.10833,185.405755
4,bbc5,3,nmf,cosine,1,173.454,0.212041,5.45356,183.119601
5,bbc5,3,nmf,correlation,1,176.339,1.767040,5.13059,187.236630
6,bbc5,4,nmf,euclidean,1,174.223,0.237139,1.82648,181.286619
7,bbc5,4,nmf,cosine,1,174.817,0.221126,1.81655,181.854676
8,bbc5,4,nmf,correlation,1,177.192,1.667060,1.59783,185.456890
9,bbc5,5,nmf,euclidean,1,175.975,0.238074,3.26700,185.480074


In [4]:
drop_query = '''DROP TABLE EXPERIMENTS CASCADE'''
cursor.execute(drop_query)
connection.commit()

In [75]:
create_results_table = '''CREATE TABLE EXPERIMENTS_RESULT
    (EXPERIMENT_ID INTEGER REFERENCES EXPERIMENTS(EXPERIMENT_ID),
    GAP_K INTEGER NOT NULL,
    GAP_K2 INTEGER NOT NULL,
    SILHOUETTE_K INTEGER NOT NULL,
    SILHOUETTE_K2 INTEGER NOT NULL,
    COHERENCE_K INTEGER NOT NULL,
    COHERENCE_K2 INTEGER NOT NULL)
    '''

cursor.execute(create_results_table)
connection.commit()

In [78]:
alter_results_table = """ALTER TABLE EXPERIMENTS_RESULT
 ADD CONSTRAINT UC_RESULT UNIQUE (EXPERIMENT_ID)"""

cursor.execute(alter_results_table)
connection.commit()